<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [17]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 


In [157]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [8]:
# считаем число строк в таблице вакансий
query_3_1 = f''' 
    select count(*)
    from public.vacancies
'''

In [19]:
# результат запроса
display(pd.read_sql_query(query_3_1, connection))

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [12]:
# считаем число строк в таблице работодателей
query_3_2 = f''' 
    select count(*)
    from public.employers
'''

In [20]:
# результат запроса
display(pd.read_sql_query(query_3_2, connection))

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [14]:
# считаем число строк в таблице регионов
query_3_3 = f''' 
    select count(*)
    from public.areas
'''

In [21]:
# результат запроса
display(pd.read_sql_query(query_3_3, connection))

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [16]:
# считаем число строк в таблице сфер деятельности
query_3_4 = f''' 
    select count(*)
    from public.industries
'''

In [22]:
# результат запроса
display(pd.read_sql_query(query_3_4, connection))

,count
0,294


***

###### выводы по предварительному анализу данных
Судя по соотношению числа работодателей к числу вакансий, у многих работодателей представлено более 1й вакансии. Также это может быть связано с дублированием вакансий на разные регионы.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [35]:
# выводим названия регионов и подсчитываем число вакансий в каждом из них
query_4_1 = f''' 
    select a.name, count(*)
    from public.vacancies v
    left join public.areas a on v.area_id = a.id
    group by 1
    order by 2 desc
'''

In [36]:
# результат запроса
display(pd.read_sql_query(query_4_1, connection))

,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [37]:
# фильтруем вакансии где "зарплата от" или "зарплата до" не пустые и подсчитываем
query_4_2 = f''' 
    select count(*)
    from public.vacancies v
    where salary_from is not null
    or salary_to is not null
    
'''

In [38]:
# результат запроса
display(pd.read_sql_query(query_4_2, connection))

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [45]:
# подсчитываем отдельно среднее для "зарплата от" и для "зарплата до", объединяем результаты
query_4_3 = f''' 
    select avg(salary_from)
    from public.vacancies 
    union
    select avg(salary_to)
    from public.vacancies 
    
'''

In [46]:
# результат запроса
display(round(pd.read_sql_query(query_4_3, connection), 0))

,avg
0,71065.0
1,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [49]:
# подсчитываем комбинации каждого типа трудоустройства и рабочего графика
query_4_4 = f''' 
    select schedule, employment, count(*)
    from public.vacancies 
    group by 1,2 
    order by 3 desc
    
'''

In [50]:
# результат запроса
display(pd.read_sql_query(query_4_4, connection))

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [51]:
# подсчитывем в скольки вакансиях требуется определённый стаж работы
query_4_5 = f''' 
    select experience, count(*)
    from public.vacancies 
    group by 1 
    order by 2 asc
    
'''

In [52]:
# результат запроса
display(pd.read_sql_query(query_4_5, connection))

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

###### выводы по детальному анализу вакансий 
- Большинство вакансий расположено в городах-миллионниках; 
- Менее половины вакансий содержат информацию о предлагаемом уровне зарплаты; 
- Полная занятость - самый популярный тип трудоустройства, независимо от графика; 
- Размер требуемого опыта соискателя в большинстве случаев ожидаемо лежит в диапазоне 1-3 года, тк работодателям удобнее, чтоб будущие сотрудники уже имели хоть какой-то опыт работы, а должности с большим опытом зачастую закрываются из уже имеющихся у компании сотрудников.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [212]:
# выводим топ-5 работодателей по количеству вакансий
query_5_1 = f''' 
    select e.name, count(*)
    from public.vacancies v
    left join public.employers e on v.employer_id = e.id
    group by 1 
    order by 2 desc
    limit 5        
'''

In [213]:
# результат запроса
display(pd.read_sql_query(query_5_1, connection))

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [70]:
# в запросе на подсчёт работодателей и вакансий в каждом регионе сразу отфильтруем регионы с 0 вакансий 
query_5_2 = f''' 
    select a.name, count(e.id) employers, count(v.id) vacancies
    from public.areas a
    left join public.employers e on a.id = e.area
    left join public.vacancies v on a.id = v.area_id
    group by 1 
    having count(v.id) = 0
    order by 2 desc
    limit 15        
'''

In [71]:
# результат запроса
display(pd.read_sql_query(query_5_2, connection))

,name,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
5,Беларусь,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [111]:
# выбираем работодателей и подсчитываем количество уникальных регионов в их вакансиях
query_5_3 = f''' 
    select employer_id, count(distinct area_id)
    from public.vacancies
    group by 1
    order by 2 desc
    '''

In [112]:
# результат запроса
display(pd.read_sql_query(query_5_3, connection))

,employer_id,count
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71
...,...,...
14901,810278,1
14902,810313,1
14903,810551,1
14904,810688,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [122]:
# фильтруем работодателей, у которых сфера деятельности не проставлена, и подсчитываем
query_5_4 = f''' 
    select count(*)
    from public.employers e
    left join public.employers_industries ei on e.id = ei.employer_id
    where ei.employer_id is null
    '''

In [123]:
# результат запроса
display(pd.read_sql_query(query_5_4, connection))

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [136]:
# подсчитываем сколько сфер указано у каждой компании, фильтруем те, у кого их 4 шт
query_5_5 = f''' 
    select e.name, count(distinct industry_id)    
    from public.employers_industries ei
    left join public.employers e on e.id = ei.employer_id
    group by 1
    having count(distinct industry_id) = 4
    order by 1
    '''
  

In [137]:
# результат запроса
display(pd.read_sql_query(query_5_5, connection))

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1127,ЮРИОН,4
1128,ЮТИП Технологии,4
1129,ЯКласс,4
1130,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [138]:
# к таблице работодателей присоединяем таблицу сфер, фильтруем по имени сферы и подсчитываем количество
query_5_6 = f''' 
    select count(*)    
    from public.employers_industries ei
    left join public.industries i on i.id = ei.industry_id
    where i.name = 'Разработка программного обеспечения'
    '''

In [139]:
# результат запроса
display(pd.read_sql_query(query_5_6, connection))

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [195]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser') # парсим страницу

table = page.find('td', class_='hlist') # нахиодим таблицу
c_links = table.find_all('a') # извлекаем ссылки с названиями городов

cities = [] # переменная для финального списка, в который добавим названия через цикл ниже
for i in range(len(c_links)): 
    cities.append(c_links[i].text)
    
cities = tuple(cities) # преобразуем список в кортеж для удобства использования в запросе
    
print(cities) 

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [202]:
# текст запроса
query_5_7 = f''' 
    select a.name, count(*)
    from public.vacancies v
    join public.areas a on a.id = v.area_id
    where a.name in {cities}
    and employer_id = 1740
    group by 1
    
    union 
    select 'total', count(*)
    from public.vacancies v
    join public.areas a on a.id = v.area_id
    where a.name in {cities}
    and employer_id = 1740
    '''

In [203]:
# результат запроса
display(pd.read_sql_query(query_5_7, connection))

,name,count
0,Уфа,26
1,Воронеж,32
2,Екатеринбург,39
3,Ростов-на-Дону,25
4,Новосибирск,35
5,Краснодар,30
6,Самара,26
7,Омск,21
8,Красноярск,23
9,Нижний Новгород,36


***

###### выводы по анализу работодателей
У топовых российских компаний представлено большое число вакансий, которые распределены по разным регионам. Лидирует и по числу вакансий и по числу присутствия в различных регионах компания Яндекс. Кроме того, нашла в списке топ-20 и своего работодателя с 122 шт вакансий =)

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [5]:
# ищем вакансии, относящиеся к данным, учитывая различные регистры 
query_6_1 = f''' 
    select count(*)
    from public.vacancies
    where name ilike '%данн%'
    or name ilike '%data%'    
'''

In [155]:
# результат запроса
display(pd.read_sql_query(query_6_1, connection))

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [70]:
# ищем более детально вакансии, учитываая различные регистры
query_6_2 = f''' 
    select count(*)
    from public.vacancies 
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%' )
     and (name ilike '%junior%' 
    or experience = 'Нет опыта'
    or employment = 'Стажировка')
'''

In [71]:
# результат запроса
display(pd.read_sql_query(query_6_2, connection))

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [68]:
# уточняем запрос для выбранных ключевых слов
query_6_3 = f''' 
    select count(*)
    from public.vacancies v
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name ilike '%ML%' and name not ilike '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%' )
     and (key_skills ilike '%SQL%'
    or key_skills ilike '%postgres%')
'''

In [69]:
# результат запроса
display(pd.read_sql_query(query_6_3, connection))

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [72]:
# уточняем запрос для нового ключевого слов
query_6_4 = f''' 
    select count(*)
    from public.vacancies v
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name ilike '%ML%' and name not ilike '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')
    and key_skills ilike '%Python%'
'''

In [73]:
# результат запроса
display(pd.read_sql_query(query_6_4, connection))

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [74]:
# выгружаем из базы список ключевых навыков, предложенный в подсказке способ подсчёта показался менее элегантным
query_6_5 = f''' 
    select key_skills
    --replace(key_skills, CHR(9), ' ') as key_skills, replace(key_skills, CHR(9), '') as ks 
    from public.vacancies 
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')
    and key_skills is not null    
'''
df = pd.read_sql_query(query_6_5, connection)

# делим строку по разделителям и подсчитываем число навыков для каждой вакансии в новый столбец
df['cnt_skills']= df['key_skills'].apply(lambda x: len(x.split('\t')))
avg_skill_numb = df['cnt_skills'].mean()

In [76]:
# результат запроса
print(f'В среднем число навыков в вакансиях для DS: {round(avg_skill_numb, 2)}')

В среднем число навыков в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [151]:
# сделаем группировку по опыту и расчёт среднего уровня зп сразу в запросе и выведем результата в переменную df
query_6_6 = f''' 
    select experience, round(avg(coalesce((salary_to + salary_from)/2, salary_from, salary_to)), 0) AS avg_s
    from public.vacancies 
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')
    and  (salary_from is not null or salary_to is not null)
    group by experience
    '''

df2 = pd.read_sql_query(query_6_6, connection)


In [152]:
# результат запроса
display(df2)

,experience,avg_s
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

###### выводы по предметному анализу
На рынке труда представлено неплохое разнообразие в вакансиях по работе с данными. Порядка 30% из них относятся к сфере DS. А 9,5% вакансий DS подходят для начинающего специалиста. 

Представленный уровень зарплат, однако, выглядит заниженным на текущем уровне рынка. Видимо, база вакансий была собрана несколько лет назад. С поправкой на этот факт зарплаты специалистов всех уровней опыта выглядят привлекательно.

# Общий вывод по проекту

In [159]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, 
# продумать варианты продолжения исследования

""" В качестве дополнительного исследования:
1) сравним средний уровень ЗП DS-специалиста опытом от 3 до 6 лет с общим уровнем ЗП специлистов с этим опытом
2) проверим представленность вакансий DS в регионах
"""

query_1 = f''' 
    select 'DS-спец.' empl_type, experience, round(avg(coalesce((salary_to + salary_from)/2, salary_from, salary_to)), 0) AS avg_s
    from public.vacancies 
    where (name ilike '%data scientist%' 
    or name ilike '%data science%' 
    or name ilike '%исследователь данных%' 
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')
    and  (salary_from is not null or salary_to is not null)
    group by experience
    having experience = 'От 3 до 6 лет'
    
    union
    select 'Общ.', experience, round(avg(coalesce((salary_to + salary_from)/2, salary_from, salary_to)), 0) AS avg_s
    from public.vacancies 
    where salary_from is not null or salary_to is not null
    group by experience
    having experience = 'От 3 до 6 лет'
'''
display(pd.read_sql_query(query_1, connection))

,empl_type,experience,avg_s
0,Общ.,От 3 до 6 лет,133211.0
1,DS-спец.,От 3 до 6 лет,243115.0


In [164]:
query_2 = f''' 
    select a.name, count(*)
    from public.vacancies v
    join public.areas a on v.area_id = a.id
    where (v.name ilike '%data scientist%' 
    or v.name ilike '%data science%' 
    or v.name ilike '%исследователь данных%' 
    or (v.name ilike '%ML%' and v.name not ilike '%HTML%')
    or v.name ilike '%machine learning%' 
    or v.name ilike '%машинн%обучен%' )
    group by 1
    order by 2 desc
'''
regions = pd.read_sql_query(query_2, connection)
print(f'Число регионов, где представлены вакансии DS: {regions.shape[0]}')
print('Топ-10 регионов по количеству вакансий DS:')
regions.head(10)

Число регионов, где представлены вакансии DS: 58
Топ-10 регионов по количеству вакансий DS:


,name,count
0,Москва,225
1,Санкт-Петербург,69
2,Новосибирск,23
3,Алматы,22
4,Нижний Новгород,20
5,Казань,19
6,Томск,11
7,Ростов-на-Дону,11
8,Минск,11
9,Пермь,8


### Общие выводы по проекту
- <font color='3499ff'>Распространённость вакансий DS</font> в общих чертах повторяет распространённость в целом представленных вакансий по всем направлениям. Хотя можно заметить интересную особенность, что, например, Минск в общей выборке занимает 3-е место, а в выборке по DS спустился на 8-е, а остальные города из Топ-5 общего списка сохранили лидирующие позиции и по вакансиям DS.
- <font color='3499ff'>Навыки работы с Python</font> требуются в 2/3 представленных вакансий, <font color='3499ff'>навыки работы с SQL/postgres</font> в более половины вакансий. Делаем вывод, что уже изученные на курсе материалы, а также материалы данного модуля являются востребованными на рынке навыками.
- Вызывает интерес, какие же навыки требуются в остальных вакансиях DS. Сделав выборку с исключением навыков SQL/postgres и Python, которую не буду приводить здесь для краткости, я вижу список вакансий программистов на языках С++/QML. Коллеги, у нас в выборку DS попадает лишнее =) в фильтре <font color='green'>and name not like '%HTML%'</font> стоит добавить также <font color='green'>and name not like '%QML%'</font>.
- Наконец, <font color='3499ff'>уровень зарплат</font> выше среднего в сравнеи с общим числом вакансий. А в случае наличия опыта работы более 3х лет специалисты DS могут претендовать на зп вдвое большую, чем средняя зп по всем вакансиям с аналогичным опытом.